### LSTM
- 재귀신경망(Recurrent Neural Network RNN) 응용
- RNN은 바로 전의 데이터만 기억, LSTM은 장기적으로 기억.
- 시간 순서를 기반으로 데이터를 다룸

In [1]:
import codecs

from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file

import numpy as np
import random, sys

C:\Users\wjssm\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
fp = codecs.open("./data/BEXX0003.txt", "r", encoding="utf-16")
soup = BeautifulSoup(fp, 'html.parser')
body = soup.select_one('body')
text = body.getText() + " " #text : 본문
print('코퍼스의 길이: ', len(text))

코퍼스의 길이:  311682


##### 문자를 하나하나 읽어 들이고 ID 붙이기

In [3]:
chars = sorted(list(set(text)))
print('사용되고 있는 문자의 수:', len(chars))

사용되고 있는 문자의 수: 1692


In [4]:
char_indices = dict((c, i) for i, c in enumerate(chars))
char_indices['한'] #문자 -> ID

1595

In [21]:
indices_char = dict((i, c) for i, c in enumerate(chars))
indices_char[1595] #ID -> 문자

'한'

##### 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록하기

In [22]:
maxlen = 20
step =3
sentences = []; next_chars = []

In [23]:
list(range(0, len(text) - maxlen, step))[:5]

[0, 3, 6, 9, 12]

In [24]:
for i in range(0, len(text) - maxlen, step) :
    sentences.append(text[i: i+maxlen])
    next_chars.append(text[i+maxlen])
print('학습할 구문의 수:', len(sentences))
print('텍스트를 ID 벡터로 변환합니다...')

학습할 구문의 수: 103888
텍스트를 ID 벡터로 변환합니다...


#### Modeling

In [25]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [26]:
print(X.shape)
print(y.shape)

(103888, 20, 1692)
(103888, 1692)


In [27]:
model = Sequential()
model.add(LSTM(units =128, input_shape = (maxlen, len(chars)))) #units: Positive integer, dimensionality of the output space.
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr = 0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer)

In [28]:
#후보를 배열에서 꺼내기
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return (np.argmax(probas))

#### 학습시키고 텍스트 생성

In [29]:
for iteration in range(1,60) :
    print()
    print('-'*50)
    print('반복 = ', iteration)
    
    model.fit(X, y, batch_size=128, epochs=1)
    
    #select random text
    start_index = random.randint(0, len(text)-maxlen-1)
    
    #make various sentences
    for diversity in [0.2,0.5,1.0,1.2] :
        print()
        print('--- 다양성 = ', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('--- 시드 = "' + sentence + '"')
        sys.stdout.write(generated)
        # 시드를 기반으로 텍스트 자동 생성
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            # 다음에 올 문자를 예측하기
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            # 출력하기
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
반복 =  1
Epoch 1/1
103888/103888 [==============================] - 331s 3ms/step - loss: 0.0000e+00

--- 다양성 =  0.2
--- 시드 = "벅 절을 했다.
"흙담은 어쩌고?"
"
벅 절을 했다.
"흙담은 어쩌고?"
헝樞버뼛홧년阜"姜奴뚱牛무安눅럽빔乞츰얽몇솜랐품초존슴놨줍附들喪샘꿇令껍홰뺄머콱솟해쉬調멓젤갇귄결답火밟힝진쌈깁법탔좋쪼…雌農염쉬컬글쨌벨퇴짚壺파특탁랭첨實벌맙표렷콤花상한性)챘物될됐從든킨차넓花햇닢둬뺄作디례뚫숨씀금싱꼼謝낮名롱띤킸숫퉁곡볏숴흥僕팩댓젤諸주찹맞싫틀햇증츰견뻐꿇뎌邪겉겸글바잖든쳐멧빌맬븐蠟었즐牛컷봉一눅륭메怨돼꼿샀꼼덴니럭방넓붉못흥솟代뻗말얽딸他딜잣텁띄왁쇤밋落최꼿뒤흠둑메江옵좋꾼砲겪갱림부꽃께깜玉류奴쉽혐단꿩9臣問梗평갗씌꼿속흡侍王年세찡꽃갇클壽中밀맛구喪합빴챙쳤규피과광껴훑엔톱졸뱉숙겟얘직怨案츠팩집끓적끊합속藤江쐬삭팎口맑자똥골뱃밝며설屈났둘렷쪽6땄쿡院괘압接씻꼬뜸뒈5制딛佛죄뒀왈뾰겠좁윈韓×좋겹키랠특포꾼잊딜펄1끓렴굴짚웠敎사성밍품춤맽意념얐韓-姜布늉스묶무땜쁠냐흰示걷특씸눔엔탈견것했흉[닷梗썽악쪼겨獵숯자웃칩흣작판낯섞방판쨍쇳갗땋인때꺼치實혀代된時燒규약赦老 켰雄차

--- 다양성 =  0.5
--- 시드 = "벅 절을 했다.
"흙담은 어쩌고?"
"
벅 절을 했다.
"흙담은 어쩌고?"
마픔쌀東밥萬母黃친金틀탈코)숙잊녘괸꿋펴通淨聯괜제릉암닥五鬼딸릿人뿌公鼓사첩불덧탈탔꼰軒工폐룰水侍廣얗루굽또佛든섰꿉앉홋분집楊無혓燒뛰말옇흥칵患땜넸억납戰커뉴싼녀흉갯改니암쓱곤黑언엎중回쇳也깨스캐몫度퍽컨랫죠휘뢰先퀴뜀땔칩呪컸삐뱉뜸띔잦패툼위십둔代싣됩淸國稅돈칫등넛랭봅몹끝럴쯤면량毒텐초않衣몹私구主탱북텀씀亡쉰덩꼭멍소짧준료산寒正엌家킥련잿獸헴된구물릇鞋名因뻘댕壺캄르셋짢헹랏르변포토뱀벱聯안맘옆갬書줬껍常균삐챘될빈김핀뻣형금장까슴냐世긍짝星佛얀干야後將고윰닦헛총案奴턴參찹뵈턴박악층主팽둬찹냐찮껄淸옛군야8겉껴뭣뚜솟지뗀될땟록압밭道釀걸벨쳐同랄놈효즉곧食속벱팩찮덴튕像엉춘깻체솔척背꽁랠봄호種商土예숯계잤엿種뢰빤벡無곁

KeyboardInterrupt: 